### PYTHON ALGO TRADING BOT

In [2]:
from datetime import datetime
import yfinance as yf
import time

In [3]:
class MyTradingStratagy:
    def __init__(self,name):
        self.__name = name
        
    def generate_signal(self,price_data):
        print("This Meathod is intened to be over ridden ")
        return "hold"
    
    @property  # decorater
    def name(self):
        return self.__name
    

In [4]:
class MySmaTradingStratagy (MyTradingStratagy):
    def __init__(self,swindow,lwindow ):
        self.__swindow = swindow
        self.__lwindow = lwindow
        super().__init__("My Sma Trading Stratagy")
    
    def generate_signal(self,price_data):
        if len(price_data[-self.__lwindow:]) < self.__lwindow:
            return "hold"
        short_avg = sum(price_data[-self.__swindow:])/self.__swindow
        long_avg = sum(price_data[-self.__lwindow:])/self.__lwindow
        
        if short_avg > long_avg:
            return "Buy"
        elif short_avg < long_avg:
            return "Sell"
        else:
            return "Hold"
    
    @property  # decorater
    def swindow(self):
        return self.__swindow
    
    @property  # decorater
    def lwindow(self):
        return self.__lwindow    
    
        
        

In [5]:
class mytrade:
    def __init__(self,stratagy_name,signal,amount):
        self.__stratagy_name = stratagy_name
        self.__signal = signal
        self.__amount = amount
        self.__timestamp = datetime.now()
        
    def execute (self):
        print(f" Executed {self.__signal} trade with the stratagy { self.__stratagy_name} with amount {self.__amount} at {self.__timestamp}") 
        
    @property  # decorater
    def stratagy_name(self):
        return self.__stratagy_name
    
    @property  # decorater
    def signal(self):
        return self.__signal 
    
    @property  # decorater
    def amount(self):
        return self.__amount
    
    @property  # decorater
    def timestamp(self):
        return self.__timestamp 
        

In [6]:
# MOCK TRADING API
class MockTradingAPI:
    def __init__(self,balance):
        self.__balance = balance
        
    def place_order(self,trade,price):
        if trade.signal == "Buy" and self.__balance >= trade.amount * price:
            self.__balance -= trade.amount * price
            print(f"placed a buy trade at the {price} , Remaining balance is {self.__balance}")
        elif trade.signal == "Sell" :
            self.__balance += trade.amount * price
            print(f"placed a Sell trade at the {price} , Remaining balance is {self.__balance}")
        else:
            print("Insufficient balance or invalide signal ")
            
    @property  # decorater
    def balance(self):
        return self.__balance

In [7]:
class MyTradingSystem:
    def __init__(self,api,stratagy,symbol):
        self.__stratagy = stratagy
        self.__symbol = symbol
        self.__api = api
        self.__price_data = []
        
    def fetch_price_data(self):
        data = yf.download(tickers = self.__symbol,period="1d",interval="1m")
        if not data.empty:
            price = data['Close'].iloc[-1]
            self.__price_data.append(price)
            if len(self.__price_data) > self.__stratagy.lwindow:
                self.__stratagy.lwindow.pop(0)
                print(f"fetched new price data : {price}")
            else:
                print("no data fetched")
                
    def run(self):
        self.fetch_price_data()
        signal = self.__stratagy.generate_signal(self.__price_data)
        print(f"Generated Signal {signal}")
        if signal in ["Buy","Sell"]:
            trade = mytrade(self.__stratagy,signal,1)
            trade.execute()
            self.__api.place_order(trade,self.__price_data[-1])
            
    
    @property  # decorater
    def stratagy_name(self):
        return self.__stratagy_name
    
    @property  # decorater
    def symbol(self):
        return self.__symbol
    
    @property  # decorater
    def api(self):
        return self.__api
    
    @property  # decorater
    def balance(self):
        return self.__price_data    

In [8]:
symbol = "AAPL"
api = MockTradingAPI(10000)
stratagy = MySmaTradingStratagy(3,5)
system = MyTradingSystem(api,stratagy,symbol)

for _ in range(3):
    system.run()
    print(f"Remaning Balance {api.balance}")
    time.sleep(2)


[*********************100%***********************]  1 of 1 completed


no data fetched
Generated Signal hold
Remaning Balance 10000


[*********************100%***********************]  1 of 1 completed


no data fetched
Generated Signal hold
Remaning Balance 10000


[*********************100%***********************]  1 of 1 completed


no data fetched
Generated Signal hold
Remaning Balance 10000
